In [16]:
##Zip
L1=[1,2,3]
L2=[4,5,6]
L3=[7,8]
for i in zip(L1,L2,L3):
    print(i)

(1, 4, 7)
(2, 5, 8)


In [11]:
for i,j in zip(L1,L2):
    print(i,j)

1 4
2 5
3 6


In [9]:
a=zip(L1,L2,L3)
a

In [19]:
##map()
def sum1(a,b):
    return a+b

In [21]:
L1=[1,2,3]
L2=[5,6]  
L=[x for x in map(sum1,L1,L2) ]
print(L)


[6, 8]


In [22]:
## Filter
def even(a):
    if a%2==0: return True
    else: return False
L1=[1,2,3,8,6,11,7]
L=[ x for x in filter( even, L1) ]     # even is the function name
print( L )    # selects even values from a list


[2, 8, 6]


In [23]:
## lambda
square=lambda x : x * x
cube=lambda x : x * x * x

print( square(5) )     # prints 25
print( cube(3) )        # prints 27


25
27


In [24]:
L1=[1,2,3]
L2=[5,6,7]  
L=[ x for x in map( lambda a,b : a+b, L1 , L2 ) ]
print( L )


[6, 8, 10]


In [35]:
L1=[1,2,3,8,6,11,7]
L=[x for x in filter(lambda a:a%2==0,L1)]
L

[2, 8, 6]

In [33]:
L1=[1,2,3]
L2=[5,6,7]  
z=lambda a,b:a+b
z(5,3)

8

In [39]:
import keras

Using TensorFlow backend.


In [40]:
import tensorflow as tf

In [50]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [52]:
mnist.train.images.shape

(55000, 784)

In [53]:
import numpy as np

In [54]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11773536638929938636
]
